# Format 3000,5000,5000 exclusive data to word lists
format 3000,5000,5000_exclusve in the following formats:

* all data text british: word, type, definition, example, phonetics, cefr
* all data text usa: word, type, definition, example, phonetics, cefr
* all data + pronunciation: same as above with clickable HTML
* 2 column: word, type, definiton

All of the above grouped by cefr

In [1]:
import pandas as pd
import os

/tmp/ipykernel_158630/3026406390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
DATASET = 'oxford_3000'
#DATASET = 'oxford_5000'
#DATASET = 'oxford_5000_exclusive'
df = pd.read_pickle(f"./data/english/{DATASET}.pkl")
df.head()

,word,type,cefr,phon_br,phon_n_am,definition,example,uk,us
0,a,indefinite article,a1,/ə/,/ə/,used before countable or singular nouns referr...,a man/horse/unit,a_uk.mp3,a_us.mp3
1,abandon,verb,b2,/əˈbændən/,/əˈbændən/,"to leave somebody, especially somebody you are...","abandon somebody, The baby had been abandoned ...",abandon_uk.mp3,abandon_us.mp3
2,ability,noun,a2,/əˈbɪləti/,/əˈbɪləti/,the fact that somebody/something is able to do...,People with the disease may lose their ability...,ability_uk.mp3,ability_us.mp3
3,able,adjective,a2,/ˈeɪbl/,/ˈeɪbl/,"to have the skill, intelligence, opportunity, ...",You must be able to speak French for this job.,able_uk.mp3,able_us.mp3
4,about,adverb,a1,/əˈbaʊt/,/əˈbaʊt/,a little more or less than; a little before or...,It costs about $10.,about_uk.mp3,about_us.mp3


## HTML+PDF all columns alphabetical

In [3]:
from utils.utils import replace_word_in_example_with_underscore
from utils.dataset import load_english as load_data

In [4]:
data = load_data()

style = data.style.format(
    escape="html",
    )
style = style.hide(axis='index')

html = style.to_html()
filename = DATASET + '_alphabetical'
with open(f'output/{filename}.html', 'w') as f:
    f.write('<meta charset="UTF-8">'+html)

cmd = f'pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css '
os.system(cmd)

/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

In [5]:
# Complete to HTML
data = load_data()
data["example"] = data.apply(lambda row: replace_word_in_example_with_underscore(row.word, row.example) , axis=1)

style = data.style.format(
    escape="html",
    )
style = style.hide(axis='index')

html = style.to_html()
filename = DATASET + '_underscore_alphabetical'
with open(f'output/{filename}.html', 'w') as f:
    f.write('<meta charset="UTF-8">'+html)

cmd = f'pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css '
os.system(cmd)

/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

## HTML+PDF all columns grouped by CEFR

In [6]:
data = load_data()
data["example"] = data.apply(lambda row: replace_word_in_example_with_underscore(row.word, row.example) , axis=1)
print(data["example"][0])
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_underscore_by_cefr'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css  --title '{filename}'"""
os.system(cmd)


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


_ man/horse/unit






pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

In [7]:
data = load_data()
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css  --title '{filename}'"""
os.system(cmd)


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

## HTML+PDF all columns grouped by CEFR shuffle

In [8]:
# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    data = load_data()
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})
    data = data.sample(frac=1)

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr_shuffle'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css  --title '{filename}'"""
os.system(cmd)


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

In [9]:
# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    data = load_data()
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})
    data = data.sample(frac=1)

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr_shuffle'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css  --title '{filename}'"""
os.system(cmd)


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


pagedjs-cli not found. Please select a different --pdf-engine or install pagedjs-cli -- see also /usr/share/doc/pandoc/README.Debian


12032

## 2 Column LateX word,type and definition alphabetical

In [10]:
from utils.utils import FixLatexLine

In [11]:
# 2 Column word + definition
data = load_data()
data = data[["word", "definition"]]

style = data.style.format(
    escape="latex",
    )
style = style.hide(axis='index')
style = style.hide(axis='columns')

column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
fix_latex_line = FixLatexLine(column_format).fix_latex_line
latex = style.to_latex(
    environment='supertabular',
    column_format=column_format
)

latex_lines = latex.splitlines()
latex = '\n'.join((map(fix_latex_line, latex_lines)))

filename = DATASET + '_table_alphabetical'
with open(f'./build/{filename}.tex', 'w') as f:
    f.write(latex)

/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


## 2 Column LateX word,type and definition by CEFR

In [12]:
data = load_data()
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

/home/jelle/wd/words/src/utils/english.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


In [13]:
data_by_cefr[1].head()

,word,type,cefr,phonetics (UK),phonetics (US),definition,example
2,ability,noun,A2,/əˈbɪləti/,/əˈbɪləti/,the fact that somebody/something is able to do...,People with the disease may lose their ability...
3,able,adjective,A2,/ˈeɪbl/,/ˈeɪbl/,"to have the skill, intelligence, opportunity, ...",You must be able to speak French for this job.
8,abroad,adverb,A2,/əˈbrɔːd/,/əˈbrɔːd/,in or to a foreign country,to go/travel/live/study abroad
13,accept,verb,A2,/əkˈsept/,/əkˈsept/,to take willingly something that is offered; t...,He asked me to marry him and I accepted.
17,accident,noun,A2,/ˈæksɪdənt/,/ˈæksɪdənt/,"an unpleasant event, especially in a vehicle, ...",a car/road/traffic accident


In [14]:
for data, cefr in zip(data_by_cefr, cefrs):
    if data.empty:
        continue
    data = data[["word", "definition", "type", "cefr"]]
    data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
    data = data[["word", "definition"]]

    style = data.style.format(
        escape="latex",
        )
    style = style.hide(axis='index')
    style = style.hide(axis='columns')

    column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
    latex = style.to_latex(
        environment='supertabular',
        column_format=column_format
    )

    latex_lines = latex.splitlines()

    latex = '\n'.join((map(fix_latex_line, latex_lines)))

    filename = f'{DATASET}_{cefr}'
    with open(f'build/{filename}.tex', 'w') as f:
        f.write(latex)

/tmp/ipykernel_158630/1060421477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_158630/1060421477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_158630/1060421477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

## 2 Column LateX word,type and definition by CEFR shuffle

In [15]:
for data, cefr in zip(data_by_cefr, cefrs):
    if data.empty:
        continue
    data = data[["word", "definition", "type", "cefr"]]
    data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
    data = data[["word", "definition"]]

    data = data.sample(frac = 1)

    style = data.style.format(
        escape="latex",
        )
    style = style.hide(axis='index')
    style = style.hide(axis='columns')

    column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
    latex = style.to_latex(
        environment='supertabular',
        column_format=column_format
    )

    latex_lines = latex.splitlines()
    latex = '\n'.join((map(fix_latex_line, latex_lines)))

    filename = f'{DATASET}_shuffle_{cefr}'
    with open(f'build/{filename}.tex', 'w') as f:
        f.write(latex)

/tmp/ipykernel_158630/3056780930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_158630/3056780930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_158630/3056780930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [16]:
# Run terminal cd format latexmk oxford*.tex to finish build pdfs
cmd_build = f"latexmk -pdf -cd format/{DATASET}*.tex -outdir=../output" 
os.system(cmd_build)

Rc files read:
  /etc/LatexMk
Latexmk: This is Latexmk, John Collins, 4 Apr. 2023. Version 4.80.
Latexmk: Changing directory to 'format/'
Latexmk: Undoing directory change



------------
Latexmk: Could not find file 'oxford_3000*.tex'.
-- Use the -f option to force complete processing.


2816

In [17]:
# Clean any build files in output/
files_to_remove = ["*.fls", "*.log", "*.html", "*.toc", "*.synctex*", "*.fdb*", "*.aux"]
cmd_cleanup_output = "rm "+" ".join(f"output/{f}" for f in files_to_remove)
cmd_cleanup_format = "rm "+" ".join(f"format/{f}" for f in files_to_remove+["*.pdf"])
cmd_cleanup_main = "rm "+" ".join(f"./{f}" for f in files_to_remove)
os.system(";".join((cmd_cleanup_format, cmd_cleanup_main, cmd_cleanup_output)))

rm: cannot remove 'format/*.fls': No such file or directory
rm: cannot remove 'format/*.log': No such file or directory
rm: cannot remove 'format/*.html': No such file or directory
rm: cannot remove 'format/*.toc': No such file or directory
rm: cannot remove 'format/*.synctex*': No such file or directory
rm: cannot remove 'format/*.fdb*': No such file or directory
rm: cannot remove 'format/*.aux': No such file or directory
rm: cannot remove 'format/*.pdf': No such file or directory
rm: cannot remove './*.fls': No such file or directory
rm: cannot remove './*.log': No such file or directory
rm: cannot remove './*.html': No such file or directory
rm: cannot remove './*.toc': No such file or directory
rm: cannot remove './*.synctex*': No such file or directory
rm: cannot remove './*.fdb*': No such file or directory
rm: cannot remove './*.aux': No such file or directory
rm: cannot remove 'output/*.fls': No such file or directory
rm: cannot remove 'output/*.log': No such file or directory
r

256